In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping
import os
import joblib # Para guardar y cargar el scaler


# --- 1. Lista de tickers del IBEX 35 ---
IBEX35_TICKERS = [
    "ACX.MC", "ACS.MC", "AENA.MC", "ALM.MC", "AMS.MC", "BBVA.MC", "BKT.MC",
    "CABK.MC", "CLNX.MC", "COL.MC", "ELE.MC", "ENG.MC", "FER.MC",
    "GRF.MC", "IAG.MC", "IBE.MC", "IDR.MC", "ITX.MC", "MAP.MC",
    "MEL.MC", "MRL.MC", "MTS.MC", "NTGY.MC", "RED.MC", "REP.MC",
    "ROVI.MC", "SAB.MC", "SAN.MC", "SCYR.MC", "SLR.MC", "TEF.MC",
    "TL5.MC", "UNI.MC", "VCX.MC", "ASC.MC", "PUIG.MC", "APAM.MC"
]


# --- 2. Función para obtener datos de múltiples tickers ---
def get_historical_data_for_training_multiple(ticker_list, start_date, end_date):
    """
    Descarga datos históricos de múltiples tickers de Yahoo Finance y los consolida.
    """
    all_data = pd.DataFrame()
    for ticker_symbol in ticker_list:
        print(f"Descargando datos para: {ticker_symbol}...")
        try:
            ticker = yf.Ticker(ticker_symbol)
            data = ticker.history(start=start_date, end=end_date, interval="1d")
            if not data.empty:
                data['Ticker'] = ticker_symbol
                # reset_index() convierte el índice 'Date' en una columna normal
                all_data = pd.concat([all_data, data[['Close', 'Ticker']].reset_index()])
            else:
                print(f"Advertencia: No se encontraron datos para {ticker_symbol} en el rango.")
        except Exception as e:
            print(f"Error al obtener datos para {ticker_symbol}: {e}")

    if all_data.empty:
        print("ERROR: No se pudo recolectar datos de ningún ticker.")
        return None

    all_data.sort_values(by=['Date', 'Ticker'], inplace=True)
    return all_data

# --- 3. Crear secuencias X, y a partir de datos escalados ---
def create_sequences(data_scaled, look_back):
    """
    Crea secuencias X (entrada) y y (salida) para la RNN.
    """
    X, y = [], []
    for i in range(len(data_scaled) - look_back):
        X.append(data_scaled[i:(i + look_back), 0])
        y.append(data_scaled[i + look_back, 0])
    return np.array(X), np.array(y)

# --- 4. Script Principal para Entrenamiento ---
if __name__ == "__main__":
    print("Iniciando el entrenamiento del modelo RNN con múltiples tickers...")

    # --- Configuración de datos y modelo ---
    START_DATE = "2010-01-01" # Rango amplio para un entrenamiento robusto
    # Se usa la fecha actual de ejecución en Colab como fin de los datos históricos.
    END_DATE = pd.to_datetime('today').strftime("%Y-%m-%d")
    LOOK_BACK = 10 # Cuántos pasos de tiempo anteriores usar para predecir el siguiente
    RNN_UNITS = 50 # Número de unidades en la capa SimpleRNN
    EPOCHS = 200 # Aumentamos las epochs, pero usaremos EarlyStopping
    BATCH_SIZE = 64 # Tamaño del lote
    VALIDATION_SPLIT = 0.2 # 20% de los datos más recientes para validación

    MODEL_FILENAME = "rnn_ibex_predictor_v2.h5"
    SCALER_FILENAME = "scaler_ibex_predictor_v2.pkl"

    # --- Configuración de la ruta de guardado para Google Drive ---
    SAVE_DIRECTORY = "/content/drive/MyDrive" # <--- ¡CAMBIO AQUÍ!

    # Crear la carpeta si no existe (aunque para la raíz de Drive es redundante una vez montado)
    os.makedirs(SAVE_DIRECTORY, exist_ok=True)
    print(f"Los modelos se guardarán en: '{SAVE_DIRECTORY}'")

    # 1. Obtener los datos históricos de todos los tickers
    all_raw_data = get_historical_data_for_training_multiple(IBEX35_TICKERS, START_DATE, END_DATE)

    if all_raw_data is None or all_raw_data.empty:
        print("No se pudo proceder con el entrenamiento debido a la falta de datos consolidados.")
    else:
        # 2. Entrenar un único scaler con TODOS los precios de cierre combinados
        prices_for_scaler_fit = all_raw_data['Close'].values.reshape(-1, 1)
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(prices_for_scaler_fit)

        # 3. Preparar los datos y crear secuencias para cada ticker usando el scaler global
        X_combined, y_combined = [], []

        for ticker_symbol in all_raw_data['Ticker'].unique():
            ticker_data_raw = all_raw_data[all_raw_data['Ticker'] == ticker_symbol].sort_values(by='Date')['Close'].values.reshape(-1, 1)

            # Transformar los datos del ticker usando el scaler GLOBAL
            ticker_data_scaled = scaler.transform(ticker_data_raw)

            # Crear secuencias (X) y etiquetas (y) para este ticker
            if len(ticker_data_scaled) >= LOOK_BACK + 1:
                X_ticker, y_ticker = create_sequences(ticker_data_scaled, LOOK_BACK)
                X_combined.extend(X_ticker)
                y_combined.extend(y_ticker)
            else:
                print(f"Advertencia: Datos insuficientes para {ticker_symbol} para crear secuencias con look_back={LOOK_BACK}. ({len(ticker_data_scaled)} puntos)")

        X_combined = np.array(X_combined)
        y_combined = np.array(y_combined)

        if X_combined.shape[0] == 0:
            print("ERROR: No se pudieron crear suficientes secuencias de datos para el entrenamiento después del preprocesamiento.")
        else:
            # Reshape para Keras: [muestras, timesteps, características]
            X_combined = np.reshape(X_combined, (X_combined.shape[0], X_combined.shape[1], 1))

            # 4. Dividir datos en entrenamiento y validación (manteniendo el orden temporal)
            split_index = int(len(X_combined) * (1 - VALIDATION_SPLIT))
            X_train, X_val = X_combined[:split_index], X_combined[split_index:]
            y_train, y_val = y_combined[:split_index], y_combined[split_index:]

            print(f"Datos de entrenamiento: {X_train.shape[0]} secuencias")
            print(f"Datos de validación: {X_val.shape[0]} secuencias")

            # 5. Construir y compilar el modelo RNN
            model = Sequential()
            model.add(SimpleRNN(units=RNN_UNITS, activation='relu', input_shape=(LOOK_BACK, 1)))
            model.add(Dense(units=1))
            model.compile(optimizer='adam', loss='mean_squared_error')

            # 6. Callbacks para el entrenamiento (EarlyStopping)
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            print(f"Entrenando el modelo con {X_train.shape[0]} secuencias...")
            history = model.fit(X_train, y_train,
                                epochs=EPOCHS,
                                batch_size=BATCH_SIZE,
                                validation_data=(X_val, y_val),
                                callbacks=[early_stopping],
                                verbose=1)

            print(f"Entrenamiento completado. Pérdida final (entrenamiento): {history.history['loss'][-1]:.4f}")
            print(f"Mejor pérdida de validación: {min(history.history['val_loss']):.4f}")

            # 7. Guardar el modelo entrenado y el scaler en Google Drive
            model_save_path = os.path.join(SAVE_DIRECTORY, MODEL_FILENAME)
            model.save(model_save_path)
            print(f"Modelo guardado en: '{model_save_path}'")

            scaler_save_path = os.path.join(SAVE_DIRECTORY, SCALER_FILENAME)
            joblib.dump(scaler, scaler_save_path)
            print(f"Scaler guardado en: '{scaler_save_path}'")

    print("Proceso de entrenamiento finalizado.")

Iniciando el entrenamiento del modelo RNN con múltiples tickers...
Los modelos se guardarán en: '/content/drive/MyDrive'
Descargando datos para: ACX.MC...
Descargando datos para: ACS.MC...
Descargando datos para: AENA.MC...
Descargando datos para: ALM.MC...
Descargando datos para: AMS.MC...
Descargando datos para: BBVA.MC...
Descargando datos para: BKT.MC...
Descargando datos para: CABK.MC...
Descargando datos para: CLNX.MC...
Descargando datos para: COL.MC...
Descargando datos para: ELE.MC...
Descargando datos para: ENG.MC...
Descargando datos para: FER.MC...
Descargando datos para: GRF.MC...
Descargando datos para: IAG.MC...
Descargando datos para: IBE.MC...
Descargando datos para: IDR.MC...
Descargando datos para: ITX.MC...
Descargando datos para: MAP.MC...
Descargando datos para: MEL.MC...
Descargando datos para: MRL.MC...
Descargando datos para: MTS.MC...
Descargando datos para: NTGY.MC...
Descargando datos para: RED.MC...
Descargando datos para: REP.MC...
Descargando datos para: 

ERROR:yfinance:$TL5.MC: possibly delisted; no timezone found


Advertencia: No se encontraron datos para TL5.MC en el rango.
Descargando datos para: UNI.MC...
Descargando datos para: VCX.MC...


ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:$VCX.MC: possibly delisted; no timezone found
ERROR:yfinance:$ASC.MC: possibly delisted; no timezone found


Advertencia: No se encontraron datos para VCX.MC en el rango.
Descargando datos para: ASC.MC...
Advertencia: No se encontraron datos para ASC.MC en el rango.
Descargando datos para: PUIG.MC...
Error al obtener datos para PUIG.MC: Length mismatch: Expected axis has 2 elements, new values have 1 elements
Descargando datos para: APAM.MC...
Datos de entrenamiento: 99184 secuencias
Datos de validación: 24796 secuencias


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Entrenando el modelo con 99184 secuencias...
Epoch 1/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 2.8582e-05 - val_loss: 5.5170e-05
Epoch 2/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 1.7981e-06 - val_loss: 4.7369e-05
Epoch 3/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.7133e-06 - val_loss: 5.5095e-05
Epoch 4/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 1.5701e-06 - val_loss: 4.5272e-05
Epoch 5/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 1.5331e-06 - val_loss: 4.5504e-05
Epoch 6/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 1.4597e-06 - val_loss: 4.8870e-05
Epoch 7/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.5208e-06 - val_loss: 4.5754e-05
Epoch 8/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 1.4749e-06 - val_loss: 5.3187e-05
Epoch 9/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 1.5321e-06 - val_loss: 4.6879e-05
Epoch 10/200
1550/1550 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 1.4484e-06 - val

Entrenamiento completado. Pérdida final (entrenamiento): 0.0000
Mejor pérdida de validación: 0.0000
Modelo guardado en: '/content/drive/MyDrive/rnn_ibex_predictor_v2.h5'
Scaler guardado en: '/content/drive/MyDrive/scaler_ibex_predictor_v2.pkl'
Proceso de entrenamiento finalizado.
